In [1]:
import sys
import os
import gc
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np

path_build = "../../data/tencent2020/build/"
path_save = "../../data/tencent2020/save/"

pickle_path = "../../data/tencent2020/pickle"
if not os.path.exists(pickle_path):
    os.mkdir(pickle_path) 
pd.set_option('display.max_columns', None)

In [2]:
train_click_log = pd.read_csv(path_save + "train_click_log.csv", encoding="utf-8")
test_click_log = pd.read_csv(path_save + "test_click_log.csv", encoding="utf-8")
train_click_log.shape, test_click_log.shape

((30082771, 9), (33585512, 9))

train：user_id 范围 (1, 900000)  
test ：user_id 范围 (3000001, 4000000) 

In [3]:
click_log = train_click_log.append(test_click_log)
click_log.sort_values(by=['user_id','time'], ascending=[True,True], inplace=True)
click_log = click_log.reset_index(drop=True)
click_log = click_log[click_log['click_times']<=8] #去除点击次数99.99%外的数据记录
print(click_log.shape)
click_log.head()

(63663353, 9)


,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,20,1,821396,1,724607,0,5,7293,326
1,20,1,209778,1,188507,136,2,9702,6
2,20,1,877468,1,773445,0,5,29455,106
3,39,1,1683713,1,1458878,0,5,14668,326
4,40,1,122032,1,109959,1334,2,11411,0


## 统计特征生成

In [4]:
# 统计ctr特征
user_ids = pd.read_pickle(f"{pickle_path}/user_ids_relencode.pickle")
# base_features = ['creative_id', 'ad_id']
# dense_features = []
# for fea in base_features:
#     for g in [1]:
#         dense_features.append(f'mean_{fea}_gender_{g}')
#         dense_features.append(f'sum_{fea}_gender_{g}')
#     for a in [1,2,3,4,5,6,7,8,9,10]:
#         dense_features.append(f'mean_{fea}_age_{a}')
#         dense_features.append(f'sum_{fea}_age_{a}')
# user_ids = user_ids.drop(dense_features, axis=1)      
# print(user_ids.columns) 
# user_ids.to_pickle(f"{pickle_path}/user_ids_relencode.pickle")


In [5]:
user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
ctr_df = train_click_log.merge(user, how='left', on='user_id')
ctr_df = pd.concat([ctr_df, pd.get_dummies(ctr_df['age'],prefix='age'), pd.get_dummies(ctr_df['gender'],prefix='gender')], axis=1)
del ctr_df['gender_2']
print(ctr_df.head())


gender_p = dict(user['gender'].value_counts() / user_ids.shape[0])
age_p = dict(user['age'].value_counts() / user_ids.shape[0])
print(gender_p, age_p)

ctr_list = ['gender_1', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8', 'age_9', 'age_10']
del user
gc.collect()

   time  user_id  creative_id  click_times    ad_id  product_id  \
0     9    30920       567330            1   504423       30673   
1    65    30920      3072255            1  2642300        1261   
2    56    30920      2361327            1  2035918        1261   
3     6   309204       325532            1   292523       27081   
4    59   309204      2746730            1  2362208           0   

   product_category  advertiser_id  industry  age  gender  age_1  age_2  \
0                 3          32638       319    2       1      0      1   
1                 2           6783         6    2       1      0      1   
2                 2           6783         6    2       1      0      1   
3                 3          32066       242    6       1      0      0   
4                18          14682        88    6       1      0      0   

   age_3  age_4  age_5  age_6  age_7  age_8  age_9  age_10  gender_1  
0      0      0      0      0      0      0      0       0         1  
1   

63

In [6]:
#numeric_features = ['creative_id', 'ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry']
numeric_features = ['ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry']

for fea in numeric_features:
    print(fea)
    creative_id_ctr = ctr_df.drop_duplicates(['user_id',fea])

    creative_id_ctr_df = creative_id_ctr.groupby([fea])[ctr_list].mean().add_prefix(f'mean_{fea}_').reset_index()
    creative_id_sum = creative_id_ctr.groupby([fea])[ctr_list].sum().add_prefix(f'sum_{fea}_').reset_index()
    creative_id_size = creative_id_ctr.groupby([fea]).size().reset_index().rename({0:'size'}, axis=1)
    creative_id_ctr_df[f'size'] = creative_id_size['size']
    #贝叶斯平滑
    for f in ctr_list:
        M = creative_id_ctr_df[f'mean_{fea}_{f}'].mean()
        S = creative_id_ctr_df[f'mean_{fea}_{f}'].var()
        alpha =( M * ((M*(1-M))/(S)-1)  )*10
        beta  =( (1-M) * ((M*(1-M))/(S)-1) )*10
        creative_id_ctr_df[f'sum_{fea}_{f}'] = creative_id_sum[f'sum_{fea}_{f}']
        creative_id_ctr_df[f'mean_{fea}_{f}'] = (creative_id_ctr_df[f'sum_{fea}_{f}'] + alpha) / (creative_id_ctr_df[f'size'] + alpha + beta)


    click_log = click_log.merge(creative_id_ctr_df, how='left', on=fea)

    #填充缺失值
    click_log[f'mean_{fea}_gender_1'] = click_log[f'mean_{fea}_gender_1'].fillna(gender_p[1])
    click_log[f'sum_{fea}_gender_1'] = click_log[f'sum_{fea}_gender_1'].fillna(0)
    click_log[f'size'] = click_log['size'].fillna(0)
    for i in range(1,11):
        click_log[f'mean_{fea}_age_{i}'] = click_log[f'mean_{fea}_age_{i}'].fillna(age_p[i])
        click_log[f'sum_{fea}_age_{i}'] = click_log[f'sum_{fea}_age_{i}'].fillna(0)

    creative_id = click_log.groupby(['user_id'])[ [f'mean_{fea}_'+i for i in ctr_list]+[f'sum_{fea}_'+i for i in ctr_list] ].sum().reset_index()
    #user_ids = pd.concat([user_ids, creative_id.drop(['user_id'], axis=1)], axis=1)
    creative_id.to_pickle(f"{pickle_path}/{fea}_ctr_df.pickle")
    del creative_id_sum,creative_id_size,creative_id_ctr_df
    gc.collect()
    #user_ids = user_ids.merge(creative_id, how='left', on='user_id')
#user_ids.head()
#user_ids.to_pickle(f"{pickle_path}/user_ids_relencode.pickle")

user_ids.head()

ad_id
product_id


KeyboardInterrupt: 

In [7]:
creative_id = pd.read_pickle(f"{pickle_path}/creative_id_ctr_df.pickle")
ad_id = pd.read_pickle(f"{pickle_path}/ad_id_ctr_df.pickle")
reduce_mem_usage(creative_id)
reduce_mem_usage(ad_id)
user_ids = pd.concat([user_ids, creative_id.drop(['user_id'], axis=1), ad_id.drop(['user_id'], axis=1)], axis=1)
user_ids.to_pickle(f"{pickle_path}/user_ids_ctr.pickle")

Mem. usage decreased to 130.46 Mb (60.9% reduction)
Mem. usage decreased to 130.46 Mb (60.9% reduction)


In [13]:
#user_ids = pd.concat([user_ids, creative_id.drop(['user_id'], axis=1)], axis=1)
#creative_id2 = pd.read_pickle(f"{pickle_path}/creative_id_ctr_df.pickle")
#user_ids = pd.concat([user_ids, creative_id2.drop(['user_id'], axis=1)], axis=1)
#user_ids.to_pickle(f"{pickle_path}/user_ids_relencode.pickle")

#user_ids.head()
user_ids.columns

Index(['user_id', 'creative_id', 'ad_id', 'product_id', 'product_category',
       'advertiser_id', 'industry', 'click_times', 'time', 'creative_id_times',
       'ad_id_times', 'product_id_times', 'advertiser_id_times',
       'product_category_times', 'industry_times', 'creative_id_len',
       'ad_id_len', 'product_id_len', 'product_category_len', 'advertiser_len',
       'industry_len', 'mean_clicktimes', 'max_clicktimes', 'min_clicktimes',
       'mean_time', 'max_time', 'min_time', 'time_len', 'time_clicktimes',
       'time_creativeids', 'mean_ad_id_gender_1', 'mean_ad_id_age_1',
       'mean_ad_id_age_2', 'mean_ad_id_age_3', 'mean_ad_id_age_4',
       'mean_ad_id_age_5', 'mean_ad_id_age_6', 'mean_ad_id_age_7',
       'mean_ad_id_age_8', 'mean_ad_id_age_9', 'mean_ad_id_age_10',
       'sum_ad_id_gender_1', 'sum_ad_id_age_1', 'sum_ad_id_age_2',
       'sum_ad_id_age_3', 'sum_ad_id_age_4', 'sum_ad_id_age_5',
       'sum_ad_id_age_6', 'sum_ad_id_age_7', 'sum_ad_id_age_8',
       's

In [6]:
user_ids.columns
user_ids.to_pickle(f"{pickle_path}/user_ids_relencode.pickle")
#creative_id.shape

In [15]:
columns = click_log.columns
percent_missing = click_log.isnull().sum() * 100 / len(click_log)
missing_value_df = pd.DataFrame({'column_name': columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
time,time,0.0
user_id,user_id,0.0
creative_id,creative_id,0.0
click_times,click_times,0.0
ad_id,ad_id,0.0
product_id,product_id,0.0
product_category,product_category,0.0
advertiser_id,advertiser_id,0.0
industry,industry,0.0
mean_creative_id_gender_1,mean_creative_id_gender_1,0.0


In [4]:
time_df = click_log.groupby(['user_id','time'])['click_times'].sum().reset_index()
time_df = multi_column_LabelEncoder(time_df,columns=['click_times'])
time_df = time_df.groupby('user_id')['click_times'].apply(lambda x: list(x)).reset_index()

ad_df = click_log.groupby(['user_id','time']).size().reset_index()
ad_df.columns = ['user_id','time','id_size']
ad_df = multi_column_LabelEncoder(ad_df,columns=['id_size'])
ad_df = ad_df.groupby('user_id')['id_size'].apply(lambda x: list(x)).reset_index()

user_ids = pd.read_pickle(f"{pickle_path}/user_ids_relencode.pickle")
user_ids['time_clicktimes'] = time_df['click_times']
user_ids['time_creativeids'] = ad_df['id_size']
user_ids.to_pickle(f"{pickle_path}/user_ids_relencode.pickle")
user_ids.head()


click_times LabelEncoder......
LabelEncoder Successfully!


,user_id,click_times
0,1,"[3, 1, 1, 1, 1, 1, 1, 2, 1, 2]"
1,2,"[1, 1, 1, 1, 3, 2, 1, 1, 3, 2, 1, 4, 1, 4, 1, ..."
2,3,"[1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, ..."
3,4,"[1, 1, 1, 1, 4, 1, 1, 2, 1, 1, 6, 1, 2, 2, 4]"
4,5,"[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, ..."


In [9]:
#click_log['time_7'] = click_log['time']//7
# click_log['creative_id_times'] = click_log['creative_id']*10000000 + click_log['click_times']
# click_log['ad_id_times'] = click_log['ad_id']*10000000 + click_log['click_times']
# click_log['product_id_times'] = click_log['product_id']*10000000 + click_log['click_times']
# click_log['advertiser_id_times'] = click_log['advertiser_id']*10000000 + click_log['click_times']
# click_log['product_category_times'] = click_log['product_category']*10000000 + click_log['click_times']
# click_log['industry_times'] = click_log['industry']*10000000 + click_log['click_times']


click_log['creative_id_times'] = click_log['creative_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
click_log['ad_id_times'] = click_log['ad_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
click_log['product_id_times'] = click_log['product_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
click_log['advertiser_id_times'] = click_log['advertiser_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
click_log['product_category_times'] = click_log['product_category'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
click_log['industry_times'] = click_log['industry'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')


In [10]:
click_log = multi_column_LabelEncoder(click_log, 
                                      columns=['creative_id','ad_id', 'product_id', 'product_category', 'advertiser_id','industry' ,
                                               'creative_id_times','ad_id_times','product_id_times',
                                               'advertiser_id_times','product_category_times', 'industry_times',
                                               
                                              ])
# click_log = multi_column_LabelEncoderOrder(click_log, 
#                                       columns=['creative_id_times','ad_id_times','product_id_times',
#                                                'advertiser_id_times','product_category_times', 'industry_times',   
#                                                 'creative_id_t','ad_id_t','product_id_t',
#                                                 'advertiser_id_t','product_category_t','industry_t',                                             
#                                               ], max_features=2000000)

print(click_log['creative_id'].nunique())
print(click_log['ad_id'].nunique())
print(click_log['product_id'].nunique())
print(click_log['product_category'].nunique())
print(click_log['advertiser_id'].nunique())
print(click_log['industry'].nunique())
print(click_log['creative_id_times'].nunique())
print(click_log['ad_id_times'].nunique())
print(click_log['product_id_times'].nunique())
print(click_log['advertiser_id_times'].nunique())
print(click_log['product_category_times'].nunique())
print(click_log['industry_times'].nunique())

# print(click_log['creative_id_t'].nunique())
# print(click_log['ad_id_t'].nunique())
# print(click_log['product_id_t'].nunique())
# print(click_log['advertiser_id_t'].nunique())
# print(click_log['product_category_t'].nunique())
# print(click_log['industry_t'].nunique())

creative_id LabelEncoder......
ad_id LabelEncoder......
product_id LabelEncoder......
product_category LabelEncoder......
advertiser_id LabelEncoder......
industry LabelEncoder......
creative_id_times LabelEncoder......
ad_id_times LabelEncoder......
product_id_times LabelEncoder......
advertiser_id_times LabelEncoder......
product_category_times LabelEncoder......
industry_times LabelEncoder......
LabelEncoder Successfully!
3412767
3027358
39057
18
57870
332
4061140
3659810
57398
104101
113
1690


In [11]:
user_ids = click_log.groupby(['user_id']).agg({'creative_id':lambda x:list(x),
                                                        'ad_id':lambda x:list(x),
                                                        'product_id':lambda x:list(x),
                                                        'product_category':lambda x:list(x),
                                                        'advertiser_id':lambda x:list(x),
                                                        'industry':lambda x:list(x),   
                                                        'click_times':lambda x:list(x),
                                                         'time':lambda x:list(x),
                                                'creative_id_times':lambda x:list(x),
                                                'ad_id_times':lambda x:list(x),
                                                'product_id_times':lambda x:list(x),
                                                'advertiser_id_times':lambda x:list(x),
                                                'product_category_times':lambda x:list(x),
                                                'industry_times':lambda x:list(x),
#                                                'creative_id_t':lambda x:list(x),
#                                                 'ad_id_t':lambda x:list(x),
#                                                 'product_id_t':lambda x:list(x),
#                                                 'advertiser_id_t':lambda x:list(x),
#                                                 'product_category_t':lambda x:list(x),
#                                                 'industry_t':lambda x:list(x),
                                                       }).reset_index()

In [12]:
user_ids['creative_id_len'] = user_ids['creative_id'].map(lambda x: len(set(x)))
user_ids['ad_id_len'] = user_ids['ad_id'].map(lambda x: len(set(x)))
user_ids['product_id_len'] = user_ids['product_id'].map(lambda x: len(set(x)))
user_ids['product_category_len'] = user_ids['product_category'].map(lambda x: len(set(x)))
user_ids['advertiser_len'] = user_ids['advertiser_id'].map(lambda x: len(set(x)))
user_ids['industry_len'] = user_ids['industry'].map(lambda x: len(set(x)))
user_ids['mean_clicktimes'] = user_ids['click_times'].map(lambda x: np.mean(x))
user_ids['max_clicktimes'] = user_ids['click_times'].map(lambda x: np.max(x))
user_ids['min_clicktimes'] = user_ids['click_times'].map(lambda x: np.min(x))

user_ids['mean_time'] = user_ids['time'].map(lambda x: np.mean(x))
user_ids['max_time'] = user_ids['time'].map(lambda x: np.max(x))
user_ids['min_time'] = user_ids['time'].map(lambda x: np.min(x))

user_ids['time_len'] = user_ids['time'].map(lambda x: len(set(x)))

user_ids.to_pickle(f"{pickle_path}/user_ids_relencode.pickle")
user_ids.head(2)


,user_id,creative_id,ad_id,product_id,product_category,advertiser_id,industry,click_times,time,creative_id_times,ad_id_times,product_id_times,advertiser_id_times,product_category_times,industry_times,creative_id_len,ad_id_len,product_id_len,product_category_len,advertiser_len,industry_len,mean_clicktimes,max_clicktimes,min_clicktimes,mean_time,max_time,min_time,time_len
0,1,"[642275, 170988, 685687, 1307085, 100774, 5953...","[581171, 156554, 620126, 1166887, 92441, 55685...","[1, 125, 1, 1, 1132, 1, 1, 1, 1, 1244, 55, 55,...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[6411, 8522, 26781, 12996, 10113, 13007, 15341...","[323, 7, 104, 323, 1, 323, 73, 214, 64, 235, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]","[20, 20, 20, 39, 40, 43, 46, 52, 60, 64, 64, 7...","[3872964, 1123872, 3935014, 706993, 226871, 37...","[3358402, 950010, 3415179, 499291, 107496, 328...","[1, 6399, 1, 1, 5954, 1, 1, 1, 1, 8147, 52204,...","[99263, 103509, 38769, 8445, 2750, 8484, 13490...","[82, 59, 82, 82, 59, 51, 82, 82, 51, 59, 59, 5...","[1309, 1509, 41, 1309, 1, 1309, 1570, 634, 154...",12,12,6,3,12,9,1.076923,2,1,47.461538,76,20,10
1,2,"[52634, 128035, 32734, 478744, 10372, 349730, ...","[49388, 117016, 31581, 435462, 11432, 318236, ...","[76, 69, 118, 118, 1193, 76, 1, 1064, 1, 1, 10...","[2, 2, 2, 2, 2, 2, 18, 2, 18, 18, 2, 18, 2, 18...","[20681, 9438, 16619, 23545, 705, 20681, 31504,...","[315, 235, 7, 7, 314, 315, 47, 7, 47, 47, 239,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10, 11, 14, 17, 28, 28, 28, 38, 38, 39, 41, 4...","[3670932, 575824, 3013599, 3644043, 316325, 34...","[3202365, 430643, 2970933, 3151275, 488750, 29...","[54699, 53814, 5157, 5157, 7149, 54699, 1, 462...","[25317, 1326, 16287, 31864, 99902, 25317, 4850...","[59, 59, 59, 59, 59, 59, 51, 59, 51, 51, 59, 5...","[1252, 780, 1509, 1509, 1244, 1252, 1442, 1509...",42,42,20,3,36,15,1.022222,2,1,53.822222,90,10,28
